In [1]:
using Revise

In [2]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using Flux
using ProgressMeter
using MLDataPattern
using JLD2
using Plots; gr()
using LCPSim
using LearningMPC
using LearningMPC.Models
using DataFrames
using Blink
using RigidBodyDynamics

In [3]:
robot = BoxAtlas()
mvis = MechanismVisualizer(robot)
# IJuliaCell(mvis)
open(mvis, Blink.Window())

Blink.AtomShell.Window(1, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 4199`, ProcessRunning), TCPSocket(RawFD(51) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(1, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(56) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 1, Nullable{Any}(true))))

In [4]:
@load "../2018-05-08-box-atlas-longer-miqp/boxatlas.jld2" datasets net lqrsol mpc_params
mpc_params_data = mpc_params
mpc_params = MPCParams(robot)
mpc_params.Δt = mpc_params_data["Δt"]
mpc_params.horizon = mpc_params_data["horizon"]
net_cost = LearningMPC.LearnedCost(lqrsol, net)
net_mpc_params = MPCParams(robot)
net_mpc_params.horizon = 1
net_mpc_controller = MPCController(robot, net_mpc_params, net_cost, [lqrsol]);
full_mpc_controller = MPCController(robot, mpc_params, lqrsol, [lqrsol, net_mpc_controller]);
lqr_mpc_controller = MPCController(robot, (p = MPCParams(robot); p.horizon=1; p), lqrsol, [lqrsol])

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


(::MPCController) (generic function with 1 method)

In [5]:
x0 = nominal_state(robot)
x_init = nominal_state(robot)
set_velocity!(x0, findjoint(mechanism(robot), "floating_base"), [-0.5, 0, 0])
results = LCPSim.simulate(x0, lqr_mpc_controller, environment(robot), 0.01, 150, 
mpc_params.lcp_solver);

In [6]:
playback(mvis, results, 0.05)

In [ ]:
table = LearningMPC.run_evaluations(
    lqr_mpc_controller,
    "lqr_mpc",
    robot,
    lqrsol,
    [(1, [0])],
    [(1, linspace(-5, 5, 11)), (3, linspace(-10, 10, 11))];
)

append!(table, 
    LearningMPC.run_evaluations(
    net_mpc_controller,
    "net_mpc",
    robot,
    lqrsol,
    [(1, [0])],
    [(1, linspace(-5, 5, 11)), (3, linspace(-10, 10, 11))];
)
)

table

In [ ]:
jldopen("evaluation_table.jld2", "w") do file
    file["evaluations"] = table
end

In [ ]:
plt = plot(legend=false)
for row in eachrow(table)
    if row[:controller] == "net_mpc"
        success = -π/4 <= row[:qf][3] <= π/4
        scatter!(plt, [row[:v0][1]], [row[:v0][3]], color = "green", marker = success ? :dot : :x)
    end
end
plt

In [ ]:
plt = plot(legend=false)
for row in eachrow(table)
    if row[:controller] == "lqr_mpc"
        success = -π/4 <= row[:qf][3] <= π/4
        scatter!(plt, [row[:v0][1]], [row[:v0][3]], color = "green", marker = success ? :dot : :x)
    end
end
plt